<hr>

<table style="width:100%">
  <tr>
    <th><img align="center" src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/UNAL_Aplicación_Medell%C3%ADn.svg/1280px-UNAL_Aplicación_Medell%C3%ADn.svg.png" width="300"/></th>
    <th><img align="center" src="http://www.redttu.edu.co/es/wp-content/uploads/2016/01/itm.png" width="300"/> </th> 
    <th><img align="center" src="https://www.cienciasdelaadministracion.uns.edu.ar/wp-content/themes/enlighten-pro/images/logo-uns-horizontal.png" width="300"/></th>
  </tr>
</table>


<hr>

#### Pedro Atencio Ortiz - 2019 (pedroatencio@itm.edu.co)

# Módulo 2.2. Aplicaciones

En este notebook abordaremos los siguientes tópicos:

1. Clasificación y Regresión.
2. Redes convolutivas.
3. Redes recurrentes
4. Utilizando una red profunda pre-entrenada.
5. Deep features / Latent spaces.
6. Fine-tuning: Utilizar una red pre-entrenada y afinarla para que trabaje con nuestros datos.

In [ ]:
# Funciones utilitarias

import numpy as np
import sklearn
from sklearn import datasets
import matplotlib.pyplot as plt

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

def generate_data(data_type, noise=0.2, num_samples=200):
    
    np.random.seed(0)
    if data_type == 'moons':
        X, Y = datasets.make_moons(num_samples, noise=noise)
    elif data_type == 'circles':
        X, Y = sklearn.datasets.make_circles(num_samples, noise=noise)
    elif data_type == 'blobs':
        X, Y = sklearn.datasets.make_blobs(centers=2, cluster_std=noise)
    return X, Y

def visualize_model(model, X, Y, output='truncate'):
    XT = np.copy(X)
    # Set min and max values and give it some padding
    x_min, x_max = XT[:, 0].min() - .5, XT[:, 0].max() + .5
    y_min, y_max = XT[:, 1].min() - .5, XT[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    if(output=='truncate'):
        Z = np.round(model.predict(np.c_[xx.ravel(), yy.ravel()]))
    elif(output=='same'):
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    else:
        print("output param must be either truncate or same")
        return False

    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.figure(figsize=(7,5))
    plt.contourf(xx, yy, Z, cmap=plt.cm.bone)

    color = ['blue' if y == 1 else 'red' for y in np.squeeze(Y)]
    plt.scatter(X[:,0], X[:,1], color=color)

    plt.show()

<hr>

## 1. Clasificación y Regresión.

Las redes neuronales pueden ser configuradas para resolver problemas de clasificación (salida==etiqueta) o de regresión (salida==valor). A continuación analizaremos dos ejemplos, uno para cada caso y discutiremos las configuraciones necesarias.

<hr>

### Clasificación

Trabajemos sobre el dataset fashion-mnist tomando cada imagen como un array de valores.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, LeakyReLU, Activation
from tensorflow.keras.optimizers import RMSprop, Adam

import matplotlib.pyplot as plt

In [ ]:
mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

#Transformamos las etiquetas de salida a one-hot encoding.
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

x_train = x_train / 255.0
x_test = x_test / 255.0

print("Total de imagenes: ", len(x_train)+len(x_test))

In [ ]:
i = 76
plt.title("Categoria: "+str(y_train[i]))
plt.imshow(x_train[i], cmap='gray')

Utilicemos las capas tipo __Dense__ con activaciones __elu__ en las capas intermedias, y __softmax__ para la salida. También utilicemos la capa __Flatten__ para aplanar la imagen de entrada a un vector.

In [ ]:
tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

# Red neuronal
model = Sequential()
model.add(Input((28,28))) 
model.add(Flatten())
model.add(Dense(units=512, use_bias=True))
model.add(Activation(activation=LeakyReLU(alpha=0.2)))
model.add(Dense(units=10, activation='softmax', use_bias=True))

opt = RMSprop(lr=0.001)

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['acc'])

print(model.summary())

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=2)

In [ ]:
print("Error (Loss) final: %.4E"%(np.array(history.history['loss'][-1:]))) #Error final de la lista de errores
print("Precision (Accuracy) final: %.4f"%(np.array(history.history['acc'][-1:])))

plt.plot(history.history['loss'])
plt.plot(history.history['acc'])
plt.show()

In [ ]:
#Evaluemos el modelo

model.evaluate(x_test, y_test)

In [ ]:
#Lancemos una prediccion con nuestro modelo entrenado

i = 104

pred = np.round(model.predict(x_test[i].reshape(1,28,28)))
print(pred)
print(y_test[i])

<hr>

### Trabajemos

<br>

<font size=4>

1. Intente utilizar un batch_size menor y uno mayor, por ejemplo, 64 y 512. Qué puede observar?

<hr>

### Regresión

Intentemos construir un modelo que permita predecir valores de una serie de tiempo a partir de un conjunto de datos en una ventana de tiempo.

Generalmente, este tipo de problemas no tiene un dataset $(X, Y)$ asociado, sino una secuencia $X$, a partir de la cual se debe construir un dataset $(X[a,b], X[c])$, donde a,b y c son valores de tiempo y $a<b<c$.

In [ ]:
#Generemos una serie de tiempo sintetica con estacionalidad

def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Tiempo")
    plt.ylabel("Valor")
    plt.grid(False)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.1,
                    np.cos(season_time * 6 * np.pi),
                    2 / np.exp(9 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

time = np.arange(10 * 365 + 1, dtype="float32")
series = trend(time, 0.1)  
baseline = 10
amplitude = 40
slope = 0.005
noise_level = 3

# Create the series
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)

# Update with noise
series += noise(time, noise_level, seed=51)

split_time = 3000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

plt.figure(figsize=(15,3))
plot_series(time, series)

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

In [ ]:
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [ ]:
tf.keras.backend.clear_session()

model = Sequential()
model.add(Dense(units=256, input_dim=window_size, activation='elu'))
model.add(Dense(units=1, activation="linear"))

model.compile(loss='mse', 
              optimizer=tf.keras.optimizers.SGD(lr=8e-6, momentum=0.9),
             metrics=['mae'])
history = model.fit(dataset,epochs=100,verbose=2)

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
forecast = []
for time in range(len(series) - window_size):
    forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_valid, x_valid)
plot_series(time_valid, results)

### Trabajemos

1. Evalúe el modelo entrenado sobre el dataset de prueba __time_valid__ y imprima el loss y el mae.

<hr>

## 2. Redes convolutivas

Una red convolutiva es una arquitectura compuesta por dos etapas: 1) etapa convolutiva, que se encarga de realizar el feature engineering y 2) etapa fully connected, que estima las categorías a partir de las características extraidas en la primera etapa.

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
tf.keras.backend.clear_session()

#Etapa convolutiva
convModel = Sequential()
convModel.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(32,32,3)))
convModel.add(MaxPooling2D(2,2))
convModel.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
convModel.add(MaxPooling2D(2,2))

#Etapa fully connected
convModel.add(Flatten())
convModel.add(Dense(128, activation='relu'))
convModel.add(Dense(10, activation='softmax'))

In [ ]:
convModel.summary()

In [ ]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train),(x_test, y_test) = cifar10.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

#Requerimiento de la capa convolutiva input_shape = (batch_size, m, n, c)
x_train = x_train.reshape(50000, 32, 32, 3)
x_test = x_test.reshape(10000, 32, 32, 3)

#Transformamos las etiquetas de salida a one-hot encoding.
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
convModel.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

convModel.fit(x_train, y_train, epochs=5)
convModel.evaluate(x_test, y_test)

<hr>

## 3. Redes recurrentes 

Esta arquitectura de red contienen capas con neuronas que permiten tomar como entrada, salidas de la misma capa en instantes anteriores. Por ello son muy útiles para analizar series de tiempo. Ejemplos de datos que pueden tratarse como series de tiempo son: __video__, __audio__ y __texto__.

Trabajemos sobre el mismo ejemplo de series de tiempo que utilizamos en al sección 2.

In [ ]:
from tensorflow.keras.layers import LSTM, Lambda

In [ ]:
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [ ]:
tf.keras.backend.clear_session()

#Etapa recurrente
rnn_model = Sequential()

#Requerimiento de la capa recurrente input_shape = (batch_size, window_size, dimension del dato)
rnn_model.add(Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]))
rnn_model.add(LSTM(units=32, return_sequences=True))
rnn_model.add(LSTM(units=32))

#Etapa fully connected
rnn_model.add(Dense(1, activation='sigmoid'))
rnn_model.add(Lambda(lambda x: x*100.0))

opt = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)

rnn_model.compile(loss='mse',
              optimizer=opt,
              metrics=["mae"])

history = rnn_model.fit(dataset, epochs=10)

In [ ]:
forecast = []
for time in range(len(series) - window_size):
    forecast.append(rnn_model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_valid, x_valid)
plot_series(time_valid, results)

<hr>

## 4. Utilizando una red profunda pre-entrenada

Keras permite cargar modelos pre-entrenados de clasificación de imágenes, que podemos utilizar directamente en alguna aplicación.

Analicemos el modelo VGG16 entrenado para el dataset __image-net__.

<img align="center" src="https://github.com/psatencio/intro_keras/blob/master/figures/vgg16.png?raw=true" width="500"/>

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

In [ ]:
tf.keras.backend.clear_session()

base_model = VGG16(include_top=True, weights='imagenet', 
                   input_tensor=None, input_shape=None, 
                   pooling=None, classes=1000)

In [ ]:
print(base_model.summary())

In [ ]:
#https://www.cs.toronto.edu/~kriz/cifar.html
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train),(x_test, y_test) = cifar10.load_data()

In [ ]:
img = x_train[967]

plt.figure(figsize=(2,2))
plt.imshow(img)

Debido a que cada modelo pre-entrenado tiene sus propias especificaciones para las entradas, es necesario transformar nuestro conjunto de datos de acuerdo a dichas especificaciones. Para ello utilicemos las funciones de transformación propias para el modelo.

In [ ]:
from skimage.transform import resize

def preprocess_image(img, size=(224,224)):

    x = resize(img, output_shape=size)
    x = np.expand_dims(x, axis=0)
    x = x.astype(np.float64)*255.0

    x = vgg16.preprocess_input(x)

    return x

In [ ]:
img = preprocess_image(img)

In [ ]:
#lanzamos la prediccion y consultamos la categoria

pred = base_model.predict(img)
categoria = decode_predictions(pred, top=2) #mejores 2 activaciones

print(categoria)

<hr>

## 4. Deep features / Latent spaces

En deep learning es posible utilizar la activación de una capa intermedia de una red neuronal entrenada para un problema, y utilizar dicha activacion como caracteristicas de entrada para un nuevo clasificador utilizado en otro problema. Estas características reciben el nombre de __deep features__.

Analicemos el modelo VGG16 entrenado para el dataset __image-net__.

<img align="center" src="https://github.com/psatencio/intro_keras/blob/master/figures/deep_features.png?raw=true" width="500"/>

Algunas aplicaciones de las deep-features:

- Representación vectorial de objetos.
- Entrenamiento de nuevos modelos de clasificación o regresión.

In [ ]:
tf.keras.backend.clear_session()

#include_top=False se utiliza para cargar solo la parte convolutiva del modelo

base_model = VGG16(include_top=False, weights='imagenet', 
                   input_tensor=None, input_shape=None, 
                   pooling=None, classes=1000)

In [ ]:
base_model.summary()

In [ ]:
#https://www.cs.toronto.edu/~kriz/cifar.html
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train),(x_test, y_test) = cifar10.load_data()

In [ ]:
img = x_train[967]

plt.figure(figsize=(2,2))
plt.imshow(img)

In [ ]:
#Pre-procesamos la imagen para que se adapte a los requerimiento de VGG16

img = preprocess_image(img)

In [ ]:
#Obtengamos las deep-features para la imagen img

deep_feat = base_model.predict(img)
print(deep_feat.shape)

En algunos casos es posible necesitar obtener la salida de cualquier capa de la red y no necesariamente la ultima de la parte convolutiva. En este caso podemos construir un nuevo modelo haciendo referencias a las capas que necesitamos. 

In [ ]:
#cargamos el modelo completo
base_model = VGG16(include_top=True, weights='imagenet', 
                   input_tensor=None, input_shape=None, 
                   pooling=None, classes=1000)

In [ ]:
base_model.summary()

In [ ]:
#Construccion del nuevo modelo a partir del modelo anterior
#Nota: Utilizamos la forma de construccion C del notebook 2.1.
    
#x = model.layers[-3].output
x = base_model.get_layer('fc1').output

new_model = Model(inputs=base_model.input, outputs=x) #conexion del nuevo modelo

In [ ]:
print(new_model.summary())

In [ ]:
deep_feat = new_model.predict(img)
print(deep_feat.shape)

<hr>

## 5. Fine-tuning

Consiste en tomar un modelo pre-entrenado y utilizarlo para una nueva tarea. De manera general los pasos consisten en:

1. Congelar los pesos de la red hasta la capa en la que deseamos los deep-features.
2. Agregar nuevas capas al modelo que representen nuestro problema o tarea.
3. Crear un nuevo modelo que incluya las capas del modelo pre-entrenado y las nuevas capas.
4. Entrenar los pesos desde la conexión entre los deep-features y las nuevas capas.

<img align="center" src="https://github.com/psatencio/intro_keras/blob/master/figures/fine_tuning.png?raw=true" width="500"/>

Apliquemos este concepto para el problema del dataset CIFAR-10.

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

In [ ]:
tf.keras.backend.clear_session()

base_model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [ ]:
print(base_model.summary())

In [ ]:
print(len(base_model.layers))

In [ ]:
#1. Congelar los pesos de la red hasta la capa en la que deseamos los deep-features.

for layer in base_model.layers:
    layer.trainable = False

#2. Agregar nuevas capas al modelo que representen nuestro problema o tarea.
x = base_model.output
x = GlobalAveragePooling2D()(x) #toma los filtros de convolucion y promedia sus valores
x = Dense(1024, activation='relu')(x)
predicciones = Dense(10, activation='softmax', name='nueva_salida')(x) #10 categorias

#3. Crear un nuevo modelo que incluya las capas del modelo pre-entrenado y las nuevas capas.
new_model = Model(inputs=base_model.input, outputs=predicciones)

new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(new_model.summary())

In [ ]:
#https://www.cs.toronto.edu/~kriz/cifar.html
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train),(x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
img = x_train[967]

plt.figure(figsize=(2,2))
plt.imshow(img)

In [ ]:
#4. Entrenar los pesos desde la conexión entre los deep-features y las nuevas capas.
history = new_model.fit(x_train[:10000], y_train[:10000], epochs=10, verbose=1)

<hr>

### Trabajemos

En algunos casos el flujo de trabajo del fine-tuning consiste en los siguientes pasos:

1. Primer ajuste
    1. Congelar los pesos de la red hasta la capa en la que deseamos los deep-features.
    2. Agregar nuevas capas al modelo que representen nuestro problema o tarea.
    3. Crear un nuevo modelo que incluya las capas del modelo pre-entrenado y las nuevas capas.
    4. Entrenar en pocas épocas los pesos desde la conexión entre los deep-features y las nuevas capas.

2. Segundo ajuste
    1. Descongelar una o dos capas del modelo pre-entrenado, contando desde la conexión a las nuevas capas.
    2. Entrenar complementamente la(s) capa(s) descongelas y las nuevas capas.

Intentemos realizar esta implementación.